# FORMULA 1 RACE DATA


## 1. INTRODUCCIÓN

El caso Kaggle del cual dispongo se denomina FORMULA 1 RACE DATA. En el se encuentra información de la FORMULA  desde la temporada 1950 hasta la temporada 2017. Este Kaggle esta formado por varios datasets, en los cuales se puede encontrar datos de cada piloto, escuderia, resultados, etc. Aunque el caso Kaggle contenga exactamente 13 datasets, no todos seran utlitzados debido a la gran cantidad de datos que resultaria.

## 2. OBJETIVOS

Nada más ver el dataset que se me proporciono en lo primero que pensé es en intentar predecir algo relacionado con el piloto español más famoso de todos los tiempos en nuestro país, Fernando Alonso. Por lo que el trabajo girara entorno a él y en investigar y visualizar sus estadísticas a lo largo de todos estos años en la FORMULA 1. El objetivo principal de esta práctica es predecir los puntos que hara Alonso en su siguiente carrera en ciertos circuitos y para ello se necesitara varia información relacionada con sus resultados globales en los diferentes Gran Premios en los que ha participado. Además de esto, mientras se vaya avanzando, iré monstrando ciertos datos sobre la carrera de Fernando que me parezcan curiosos y que pueda obtener a partir de la información proporcionada por los diferentes datasets.

## 3. EDA (exploratory data analysis)

Primeramente añadiremos todas las librerias necesarias para realizar la práctica y cargaremos los datasets con más importancia para nuestra investigación los cuales son: results.csv y races.csv. En ellos podemos encontrar información relacionada con los resultados de todos los pilotos en todas las carreras. Más adelante se explicaran los diferentes atributos del dataset ya que este sufrira modificaciones respecto al original.

In [40]:
# afegim llibreries necessaries
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import scipy.stats
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from statistics import mean
from sklearn import svm 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import warnings 
warnings.filterwarnings('ignore')

#visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#llegim les dades del csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

#carreguem dataset 
dataset = load_dataset('datasets/results.csv')
data = dataset.values

#carreguem dataset 
dataset2 = load_dataset('datasets/races.csv')
data2 = dataset2.values

#carreguem dataset 
dataset3 = load_dataset('datasets/constructors.csv')
data3 = dataset3.values

Como en este caso, a diferencia de prácticas anteriores, tenemos varios datasets, realizaremos un join para asi unir todos ellos y tener todos los datos en un solo dataset. En este caso se han unido los datasets más relevantes: results.csv y races.csv, agrupandose por el atributo raceId. También se queria realizar un join con drivers.csv pero este causaba un error por lo que identificaremos a Fernando Alonso por su driverId el cual es 4.

In [41]:
inner_join_df1 = pd.merge(dataset, dataset2, on='raceId' , how='inner')
inner_join_df = pd.merge(inner_join_df1, dataset3, on='constructorId' , how='inner')
inner_join_df

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,circuitId,name_x,date,time_y,url_x,constructorRef,name_y,nationality,url_y,Unnamed: 5
0,1,18,1,1,22.000,1,1.000,1,1,10.000,...,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
1,5,18,5,1,23.000,3,5.000,5,5,4.000,...,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
2,25,19,5,1,23.000,8,3.000,3,3,6.000,...,2,Malaysian Grand Prix,2008-03-23,07:00:00,http://en.wikipedia.org/wiki/2008_Malaysian_Gr...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
3,27,19,1,1,22.000,9,5.000,5,5,4.000,...,2,Malaysian Grand Prix,2008-03-23,07:00:00,http://en.wikipedia.org/wiki/2008_Malaysian_Gr...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
4,49,20,5,1,23.000,5,5.000,5,5,4.000,...,3,Bahrain Grand Prix,2008-04-06,11:30:00,http://en.wikipedia.org/wiki/2008_Bahrain_Gran...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23772,23736,986,154,210,8.000,15,15.000,15,15,0.000,...,32,Mexican Grand Prix,2017-10-29,19:00:00,https://en.wikipedia.org/wiki/2017_Mexican_Gra...,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,NaN
23773,23756,987,154,210,8.000,11,15.000,15,15,0.000,...,18,Brazilian Grand Prix,2017-11-12,16:00:00,https://en.wikipedia.org/wiki/2017_Brazilian_G...,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,NaN
23774,23761,987,825,210,20.000,13,NaN,R,20,0.000,...,18,Brazilian Grand Prix,2017-11-12,16:00:00,https://en.wikipedia.org/wiki/2017_Brazilian_G...,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,NaN
23775,23772,988,154,210,8.000,16,11.000,11,11,0.000,...,24,Abu Dhabi Grand Prix,2017-11-26,13:00:00,https://en.wikipedia.org/wiki/2017_Abu_Dhabi_G...,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team,NaN


Una vez unidos los dos datasets, al solo necesitar los datos de Fernando Alonso, nos quedaremos con aquellas muestras con un driverId igual al de Fernando, es decir, 4.

In [44]:
datasetFernando = inner_join_df[inner_join_df.driverId == 4]
datasetFernando

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,circuitId,name_x,date,time_y,url_x,constructorRef,name_y,nationality,url_y,Unnamed: 5
36,370,36,4,1,1.000,2,2.000,2,2,8.000,...,1,Australian Grand Prix,2007-03-18,03:00:00,http://en.wikipedia.org/wiki/2007_Australian_G...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
38,391,37,4,1,1.000,2,1.000,1,1,10.000,...,2,Malaysian Grand Prix,2007-04-08,07:00:00,http://en.wikipedia.org/wiki/2007_Malaysian_Gr...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
41,417,38,4,1,1.000,4,5.000,5,5,4.000,...,3,Bahrain Grand Prix,2007-04-15,11:30:00,http://en.wikipedia.org/wiki/2007_Bahrain_Gran...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
43,437,39,4,1,1.000,2,3.000,3,3,6.000,...,4,Spanish Grand Prix,2007-05-13,12:00:00,http://en.wikipedia.org/wiki/2007_Spanish_Gran...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
44,457,40,4,1,1.000,1,1.000,1,1,10.000,...,6,Monaco Grand Prix,2007-05-27,12:00:00,http://en.wikipedia.org/wiki/2007_Monaco_Grand...,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9503,2838,153,4,18,21.000,18,NaN,R,18,0.000,...,11,Hungarian Grand Prix,2001-08-19,NaN,http://en.wikipedia.org/wiki/2001_Hungarian_Gr...,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9505,2864,154,4,18,21.000,20,NaN,R,22,0.000,...,13,Belgian Grand Prix,2001-09-02,NaN,http://en.wikipedia.org/wiki/2001_Belgian_Gran...,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9506,2877,155,4,18,21.000,21,13.000,13,13,0.000,...,14,Italian Grand Prix,2001-09-16,NaN,http://en.wikipedia.org/wiki/2001_Italian_Gran...,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9509,2907,156,4,18,21.000,17,NaN,R,21,0.000,...,19,United States Grand Prix,2001-09-30,NaN,http://en.wikipedia.org/wiki/2001_United_State...,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN


Una vez tenemos todas las muestras de las carreras de Fernando Alonso, eliminaremos ciertos artibutos los cuales no proporcionan ningún tipo de información útil para nuestros futuros estudios. Estos atributos son: number, position, positionText, statusId, round, date, time_y y url.
    

In [45]:
del(datasetFernando['number'])
del(datasetFernando['position'])
del(datasetFernando['positionText'])
del(datasetFernando['statusId'])
del(datasetFernando['round'])
del(datasetFernando['date'])
del(datasetFernando['time_y'])
del(datasetFernando['url_x'])
del(datasetFernando['rank'])
del(datasetFernando['time_x'])
del(datasetFernando['milliseconds'])

datasetFernando

,resultId,raceId,driverId,constructorId,grid,positionOrder,points,laps,fastestLap,fastestLapTime,fastestLapSpeed,year,circuitId,name_x,constructorRef,name_y,nationality,url_y,Unnamed: 5
36,370,36,4,1,2,2,8.000,58,20.000,01:26.3,221.178,2007,1,Australian Grand Prix,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
38,391,37,4,1,2,1,10.000,56,42.000,01:36.9,206.014,2007,2,Malaysian Grand Prix,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
41,417,38,4,1,4,5,4.000,57,46.000,01:34.4,206.346,2007,3,Bahrain Grand Prix,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
43,437,39,4,1,2,3,6.000,65,17.000,01:23.0,201.986,2007,4,Spanish Grand Prix,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
44,457,40,4,1,1,1,10.000,78,44.000,01:15.3,159.715,2007,6,Monaco Grand Prix,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9503,2838,153,4,18,18,18,0.000,37,NaN,NaN,NaN,2001,11,Hungarian Grand Prix,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9505,2864,154,4,18,20,22,0.000,0,NaN,NaN,NaN,2001,13,Belgian Grand Prix,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9506,2877,155,4,18,21,13,0.000,51,NaN,NaN,NaN,2001,14,Italian Grand Prix,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
9509,2907,156,4,18,17,21,0.000,36,NaN,NaN,NaN,2001,19,United States Grand Prix,minardi,Minardi,Italian,http://en.wikipedia.org/wiki/Minardi,NaN
